In [41]:
!pip install tenseal


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 23.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [40]:
import pandas as pd
import numpy as np
import math
import tenseal as ts  #for DE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from keras.models import Sequential, load_model
from keras.utils import plot_model
from keras.layers import Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix


ModuleNotFoundError: No module named 'seal'

In [28]:
#loading the wednesday dataset which contains LDOS data

df = pd.read_csv('Final_data.csv')
# df.drop(1676519)   #because it contains a row having differnt label

In [15]:
#data preprocessing

#replacing NAN values with 0
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# Separating the features and label
X = df.drop('Label', axis=1)
Y = df['Label']

non_float_cols = X.select_dtypes(exclude='float64').columns

# Convert non-float columns to float, handling potential errors
for col in non_float_cols:
    try:
        X[col] = pd.to_numeric(df[col], errors='coerce')
        # print(col)
    except ValueError:
        print(f"Error converting column '{col}' to float. Check for non-numeric values.")

X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)

# # Standardize the features
#i dont want to do this because it contains things like ip add ports and all
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# pca for dimensionality reduction
# pca = PCA(n_components=10)
# X_pca = pca.fit_transform(X)

# print(Y.unique()) #['attack' 'normal']

# # for i in range(len(Y)):
# #   if Y[i]==0:
# #     print(i)

#replacing the labels with 0 for non attcak and 1 for LDOS

Y.replace('normal', 0, inplace=True)
Y.replace('attack', 1, inplace=True)

print(Y.unique()) # 0 1
print(X.shape)
print(Y.shape)



[0 1]
(2171852, 27)
(2171852,)


<ipython-input-15-dc19d36f2712>:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y.replace('attack', 1, inplace=True)


In [16]:
# Step 1: Split into testing set and the remaining data
X_train_remain, X_test, Y_train_remain, Y_test = train_test_split(X, Y, test_size=300000, random_state=42, stratify=Y)

print(X_train_remain.shape)

# Step 2: Create the first training set for initially training the base model
X_large_train = X_train_remain.iloc[:371852]
Y_large_train = Y_train_remain.iloc[:371852]

# Remaining data for training the federated clients
X_remaining = X_train_remain.iloc[371852:]
Y_remaining = Y_train_remain.iloc[371852:]

print(X_remaining.shape)


# Step 3: Create 15 smaller training sets, 3 clients 5 model aggregation iterations
X_small_train_sets = []
Y_small_train_sets = []

# Split remaining data into 15 parts
for i in range(15):
    X_small_train = X_remaining.iloc[i * 100000:(i + 1) * 100000]
    Y_small_train = Y_remaining.iloc[i * 100000:(i + 1) * 100000]
    X_small_train_sets.append(X_small_train)
    Y_small_train_sets.append(Y_small_train)

print(X_small_train_sets[14].shape)

# print(X_train)
# print(y_train)

#let us reshape consodering the timestamp

X_large_train_reshaped=X_large_train.to_numpy().reshape((X_large_train.shape[0],1,X_large_train.shape[1]))
X_train_reshaped = [None]*15
# math.floor((i+1+2)/3)
for i in range(15):
  # print(i)
  X_train_reshaped[i]=X_small_train_sets[i].to_numpy().reshape((X_small_train_sets[i].shape[0],1, X_small_train_sets[i].shape[1])) #row timestamp Columns, here timestamp will have the values 1,2,3,4,5 indicating the period in which we collected the data

X_test_reshaped = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))#row timestamp Columns



(1871852, 27)
(1500000, 27)
(100000, 27)


In [17]:
def create_sequential():
  model = Sequential()
  model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
  model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
  model.add(Dense(1, activation = 'sigmoid', kernel_regularizer='l2'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

def print_accuracy_loss(model,X_test,y_test):
  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
  print("Accuracy and loss is ",accuracy, loss)

def print_confusion_matrix(model,X_test,y_test):
  y_pred = model.predict(X_test)
  y_pred_binary = (y_pred > 0.5).astype(int)
  cm = confusion_matrix(y_test, y_pred_binary)
  print("Confusion Matrix:")
  print(cm)

def plot_confusion_matrix(model,X_test, y_test, labels=['Attack', 'Normal']):
  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  cm = confusion_matrix(y_test, y_pred_classes)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot()
  plt.show()

def create_improved_sequential():
    model = Sequential()
    # First LSTM layer with Bidirectional and Dropout
    model.add(Bidirectional(LSTM(128, return_sequences=True, activation='tanh', kernel_regularizer='l2'), input_shape=(X_large_train_reshaped.shape[1], X_large_train_reshaped.shape[2])))
    model.add(Dropout(0.3))
    # Second LSTM layer
    model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
    model.add(Dropout(0.3))
    # Dense layers with Batch Normalization
    model.add(Dense(256, activation='relu', kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    optimizer = RMSprop(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)


In [29]:
#using Homomorphic Encryption (HE) and Differential Privacy (DP)
#in HE we send the encrypted weights and averging is performed on these encrypted weights and after averaging the weights are decrypted and then the model is updated with these weights
#in DP we add some noise to the weights before sending it so that the plain data is not sent, more noise we add more we corrupt the data and we can control it using epsilon. smaller the epsilon more noise is induced , even if epsilon is 1 some amount of noise in induced because we use laplacian

# Generate a context for HE
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40

#Add Noise to Weights:
def add_dp_noise(weights, epsilon=1.0):
    noise = [np.random.laplace(0, 1/epsilon, w.shape) for w in weights]
    noisy_weights = [w + n for w, n in zip(weights, noise)]
    return noisy_weights



In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import BinaryCrossentropy
# # import tensorflow_privacy as tfp  # For Differential Privacy
# from tf_encrypted.keras import Sequential  # For Homomorphic Encryption (example)
# from tf_encrypted.protocol import Pond  # For secure computation

#training the federated architecture

weightstemp=None;

#no of federated clients
remote_models_count=3
#no of federated iterations
federated_iterations=5
#number of epochs
epochs_count=1

#creating a 2D list to store the models of the federated clients and to store the weights of those models
remote_models=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]
weights_remote_models=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]

#we will use this for DP and HE
noisy_weights=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]
encrypted_weights=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]


# #first we train the base model and then save it so that we can forward the parameters
# base_model = create_sequential()
# history_base = base_model.fit(X_large_train_reshaped , Y_large_train, epochs = epochs_count,validation_split=0.2, verbose = 1)
# print("accuracy and loss before federated learning is ,")
# # print_accuracy_loss(base_model,X_test_reshaped,Y_test)
# base_model.save('base_model.h5')

count=0;

for i in range(federated_iterations):
  for j in range(remote_models_count):
    remote_models[i][j]=load_model('base_model.h5')
    adam = tf.keras.optimizers.Adam()  # Define your optimizer
    remote_models[i][j].compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])  # Recompile
    print(count)
    remote_models[i][j].fit(X_train_reshaped[count], Y_small_train_sets[count], epochs = epochs_count,validation_split=0.2, verbose = 1)
    print("accuracy and loss of",j, "client in",i," federated iteration is ")
    # print_accuracy_loss(remote_models[i][j],X_test_reshaped,Y_test)
    remote_models[i][j].save('remote_model'+str(i+1)+str(j+1)+'.h5')

    #saving the weights of the model to our weights list
    weights_remote_models[i][j]=remote_models[i][j].get_weights()




    # # Function to flatten weights for encryption
    # def flatten_weights(weights):
    #     return [w.flatten() if len(w.shape) > 1 else w for w in weights]

    # # Function to reshape decrypted weights to original dimensions
    # def reshape_weights(flattened_weights, original_weights):
    #     reshaped = []
    #     for flat_w, orig_w in zip(flattened_weights, original_weights):
    #         reshaped.append(flat_w.reshape(orig_w.shape) if len(orig_w.shape) > 1 else flat_w)
    #     return reshaped

  #   def encrypt_weights(weights, context):
  #   encrypted_weights = []
  #   for weight in weights:
  #     encrypted_weight = seal.CKKSVector(context, weight.flatten())
  #     encrypted_weights.append(encrypted_weight)
  #   return encrypted_weights

  # # Decrypt weights
  # def decrypt_weights(encrypted_weights, context):
  #   decrypted_weights = []
  #   for encrypted_weight in encrypted_weights:
  #     decrypted_weight = encrypted_weight.decrypt()
  #     decrypted_weights.append(decrypted_weight.reshape(weight.shape))
  #   return decrypted_weights


    # Adding noise to these weights
    noisy_weights[i][j] = add_dp_noise(weights_remote_models[i][j])

    # # Flattening weights and encrypting
    # encrypted_weights[i][j] = [
    #         ts.ckks_vector(context, w.flatten() if len(w.shape) > 1 else w)
    #         for w in noisy_weights[i][j]
    #     ]
    # with Pond() as protocol:
    #         encrypted_weight = protocol.define_private_variable(noisy_weights[i][j])
    #         encrypted_weights[i][j] = encrypted_weight
    # encrypted_weight = [ts.ckks_vector(context, w.flatten()) for w in noisy_weights[i][j]]
    # encrypted_weights[i][j] = encrypted_weight



    count=count+1

  # encrypted_avg_noisy_weights = [
  #       sum(client_weights) for client_weights in zip(*encrypted_weights[i])
  #   ]

  # aggregated_weights = [encrypted_vector.decrypt() for encrypted_vector in encrypted_avg_noisy_weights]
  # aggregated_weights = [w.reshape(base_model.get_weights()[idx].shape) for idx, w in enumerate(aggregated_weights)]
  # averaged_weights = []
  # for i in range(len(aggregated_weights)):
  #   temp=[]
  #   for j in range(len(aggregated_weights[i])):
  #     temp.append(aggregated_weights[i][j]/3)
  #   averaged_weights.append(temp)
  averaged_noisy_weights=[(w1 + w2 + w3) / 3 for w1, w2, w3 in zip(noisy_weights[i][0],noisy_weights[i][2],noisy_weights[i][2])]
  base_model.set_weights(averaged_noisy_weights)
  # base_model.set_weights(aggregated_weights)


  # Server-side aggregation on encrypted noisy weights
  # encrypted_avg_noisy_weights = [
  #   (encrypted_weights[i][j] + encrypted_weights[i][j] + encrypted_weights[i][j]) / 3
  # ]
  # with Pond() as protocol:
  #       avg_noisy_weights = protocol.reveal(encrypted_avg_noisy_weights)

  # encrypted_avg_noisy_weights =[]
  # avg_weight += noisy_weights[client_index][weight_index]

  # avg_noisy_weights = [
  #   encrypted_weight.decrypt() for encrypted_weight in encrypted_avg_noisy_weights
  # ]
  # # Reshape decrypted weights to match the original model weight dimensions
  # original_weights_shape = remote_models[0][0].get_weights()  # Reference for shapes
  # reshaped_weights = reshape_weights(avg_noisy_weights, original_weights_shape)

  # avg_noisy_weights = [encrypted_weight.decrypt() for encrypted_weight in encrypted_avg_noisy_weights]
  # Update the base model with the aggregated weights
  # base_model.set_weights(avg_noisy_weights)


In [65]:
# Encrypt model weights on each client
from seal import Plaintext, Ciphertext

def encrypt_weights(weights, encoder, encryptor, scale):
    encrypted_weights = []
    for w in weights:
        plain = Plaintext()
        encoder.encode(w.flatten().tolist(), scale, plain)  # Flatten weights
        cipher = Ciphertext()
        encryptor.encrypt(plain, cipher)
        encrypted_weights.append(cipher)
    return encrypted_weights
# Aggregate encrypted weights
def aggregate_encrypted_weights(encrypted_weights_list, evaluator):
    aggregated_weights = encrypted_weights_list[0]  # Start with the first client's weights
    for client_weights in encrypted_weights_list[1:]:
        for i in range(len(aggregated_weights)):
            evaluator.add_inplace(aggregated_weights[i], client_weights[i])
    return aggregated_weights
# Decrypt aggregated weights and compute the average
def decrypt_and_average_weights(aggregated_weights, num_clients, decryptor, encoder, original_shape):
    averaged_weights = []
    for cipher in aggregated_weights:
        plain = Plaintext()
        decryptor.decrypt(cipher, plain)
        decoded = encoder.decode(plain)  # Decode to original format
        averaged_weights.append(np.array(decoded).reshape(original_shape) / num_clients)
    return averaged_weights
import numpy as np

def add_differential_privacy(weights, epsilon, sensitivity):
    noisy_weights = []
    for w in weights:
        noise = np.random.laplace(0, sensitivity / epsilon, w.shape)  # Laplace noise
        noisy_weights.append(w + noise)
    return noisy_weights
# Add DP noise, encrypt weights, and send to server
noisy_weights = add_differential_privacy(model.get_weights(), epsilon=1.0, sensitivity=1.0)
encrypted_weights = encrypt_weights(noisy_weights, encoder, encryptor, scale)
# Client Workflow
weights = model.get_weights()
noisy_weights = add_differential_privacy(weights, epsilon=1.0, sensitivity=1.0)
encrypted_weights = encrypt_weights(noisy_weights, encoder, encryptor, scale)

# Server Workflow
all_encrypted_weights = [encrypted_weights_client1, encrypted_weights_client2, encrypted_weights_client3]
aggregated_weights = aggregate_encrypted_weights(all_encrypted_weights, evaluator)
averaged_weights = decrypt_and_average_weights(aggregated_weights, num_clients=3, decryptor=decryptor, encoder=encoder, original_shape=model.get_weights())

# Update central model
central_model.set_weights(averaged_weights)


ModuleNotFoundError: No module named 'seal'

In [ ]:


#training the federated architecture

weightstemp=None;

#no of federated clients
remote_models_count=3
#no of federated iterations
federated_iterations=5
#number of epochs
epochs_count=1

#creating a 2D list to store the models of the federated clients and to store the weights of those models
remote_models=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]
weights_remote_models=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]

#we will use this for DP and HE
noisy_weights=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]
encrypted_weights=[[0 for _ in range(remote_models_count)] for _ in range(federated_iterations)]


# #first we train the base model and then save it so that we can forward the parameters
# base_model = create_sequential()
# history_base = base_model.fit(X_large_train_reshaped , Y_large_train, epochs = epochs_count,validation_split=0.2, verbose = 1)
# print("accuracy and loss before federated learning is ,")
# # print_accuracy_loss(base_model,X_test_reshaped,Y_test)
# base_model.save('base_model.h5')

count=0;

for i in range(federated_iterations):
  for j in range(remote_models_count):
    remote_models[i][j]=load_model('base_model.h5')
    adam = tf.keras.optimizers.Adam()  # Define your optimizer
    remote_models[i][j].compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])  # Recompile
    print(count)
    remote_models[i][j].fit(X_train_reshaped[count], Y_small_train_sets[count], epochs = epochs_count,validation_split=0.2, verbose = 1)
    print("accuracy and loss of",j, "client in",i," federated iteration is ")
    # print_accuracy_loss(remote_models[i][j],X_test_reshaped,Y_test)
    remote_models[i][j].save('remote_model'+str(i+1)+str(j+1)+'.h5')

    #saving the weights of the model to our weights list
    weights_remote_models[i][j]=remote_models[i][j].get_weights()


    # Adding noise to these weights
    noisy_weights[i][j] = add_dp_noise(weights_remote_models[i][j])


    count=count+1


  averaged_noisy_weights=[(w1 + w2 + w3) / 3 for w1, w2, w3 in zip(noisy_weights[i][0],noisy_weights[i][2],noisy_weights[i][2])]
  base_model.set_weights(averaged_noisy_weights)


In [58]:
# print(type(aggregated_weights))
# print(type(aggregated_weights[0][0]))

# print(aggregated_weights)

# temp=[]
# for i in range(len(aggregated_weights)):
#   temp1=[]
#   for j in range(len(aggregated_weights[i])):
#     temp1.append(aggregated_weights[i][j]/3)
#   temp.append(temp1)

# print(temp)

print(type(weights_remote_models[0][0]))
print(type(noisy_weights[0][0]))
print((weights_remote_models[0][0]))
print((noisy_weights[0][0]))

print(type(encrypted_weights[0][0]))
print((encrypted_weights[0][0]))


print(type(encrypted_avg_noisy_weights[0][0]))
print((encrypted_avg_noisy_weights[0][0]))




<class 'list'>
<class 'list'>
[array([[ 2.1911293e-34, -4.9611115e-34, -5.6905498e-34, ...,
         4.9756472e-34,  4.3009460e-34, -2.8144714e-05],
       [-3.1514608e-35, -5.8278911e-34,  4.7198293e-34, ...,
        -1.9698640e-34,  4.7867567e-34,  3.6081909e-34],
       [ 2.5846086e-34, -3.6663141e-36,  5.7192662e-12, ...,
        -2.0331853e-34,  2.4909761e-34, -4.5723929e-34],
       ...,
       [-2.5231252e-34, -3.7654611e-34,  1.7974337e-34, ...,
        -5.3540379e-34, -1.8096959e-07,  7.6843043e-07],
       [ 1.1955408e-32,  8.8031890e-33,  4.9891399e-19, ...,
        -2.5777273e-34, -6.1598862e-15, -1.8375769e-34],
       [ 3.9104289e-34, -5.8816595e-34, -5.3636360e-13, ...,
         4.6852858e-34,  5.4116995e-09, -1.8957586e-08]], dtype=float32), array([[ 0.05072033,  0.02095141, -0.02087663, ..., -0.05880155,
         0.03107013,  0.01951434],
       [ 0.02888145,  0.01351273,  0.15260379, ...,  0.01925761,
        -0.00695168,  0.03352835],
       [ 0.02164936,  0.07356791

TypeError: 'CKKSVector' object is not subscriptable

In [ ]:
print("accuracy and loss after entire FL is ,")
print_accuracy_loss(base_model,X_test_reshaped,Y_test)
plot_confusion_matrix(base_model,X_test_reshaped,Y_test)
base_model.save('federated_base_model.h5')